<a href="https://colab.research.google.com/github/rponeconsultoria/portfoliorp/blob/main/Emails_Automaticos_VBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'*************************************************************
' Projeto: Envio Automatizado de E-mails de Cobrança
' Descrição: Automatiza o envio de e-mails com base em uma planilha de dados
' Autor: Rhyan Pablo
' Data: 28/12/2024
'*************************************************************

Option Explicit

Sub EnviarEmailsEmMassa()

    ' Declaração de variáveis principais
    Dim wsBase As Worksheet
    Dim i As Long
    Dim objetoOutlook As Object
    Dim emailItem As Object
    Dim emailInvalidoExibido As Boolean
    Dim corpoEmail As String
    Dim listaEmails As String

    ' Declaração de variáveis para dados da planilha
    Dim filial As String, solicitante As String, cliente As String
    Dim cod As Long, valor As String, dataAdiantamento As String
    Dim baixado As String, previsaoNF As String
    Dim emailPrimario As String, emailSecundario As String
    Dim mensagemPersonalizada As String

    ' Inicializações
    Set wsBase = ThisWorkbook.Sheets("base")
    emailInvalidoExibido = False
    i = 2 ' Início da leitura, assumindo que a linha 1 é cabeçalho

    Do While wsBase.Cells(i, 1).Value <> ""

        baixado = wsBase.Cells(i, 7).Value ' Coluna G: status "BAIXADO"

        ' Pula se o valor estiver marcado como "SIM"
        If UCase(baixado) = "SIM" Then
            i = i + 1
            GoTo ProximaLinha
        End If

        ' Coleta os dados da linha atual
        filial = wsBase.Cells(i, 1).Value
        solicitante = wsBase.Cells(i, 2).Value
        cliente = wsBase.Cells(i, 3).Value
        cod = wsBase.Cells(i, 4).Value
        valor = wsBase.Cells(i, 5).Value
        dataAdiantamento = wsBase.Cells(i, 6).Value
        previsaoNF = wsBase.Cells(i, 8).Value
        emailPrimario = Trim(wsBase.Cells(i, 9).Value)
        emailSecundario = Trim(wsBase.Cells(i, 10).Value)
        mensagemPersonalizada = wsBase.Cells(i, 11).Value

        ' Valida campos de e-mail
        If emailPrimario = "-" Or emailSecundario = "-" Then
            i = i + 1
            GoTo ProximaLinha
        End If

        ' Monta a lista de destinatários
        If InStr(emailPrimario, ";") > 0 Then
            listaEmails = Join(Split(emailPrimario, " "), ";")
        Else
            listaEmails = emailPrimario
        End If

        ' Validação de e-mails (requer função externa IsValidEmail)
        If Not IsValidEmail(listaEmails) Then
            If Not emailInvalidoExibido Then
                MsgBox "E-mail inválido detectado: " & listaEmails, vbExclamation
                emailInvalidoExibido = True
            End If
            i = i + 1
            GoTo ProximaLinha
        End If

        If emailSecundario <> "" And Not IsValidEmail(emailSecundario) Then
            If Not emailInvalidoExibido Then
                MsgBox "E-mail secundário inválido: " & emailSecundario, vbExclamation
                emailInvalidoExibido = True
            End If
            i = i + 1
            GoTo ProximaLinha
        End If

        ' Cria o objeto Outlook e prepara o e-mail
        Set objetoOutlook = CreateObject("Outlook.Application")
        Set emailItem = objetoOutlook.CreateItem(0)

        ' Monta o corpo do e-mail
        corpoEmail = "Mensagem de Cobrança:" & vbCrLf & _
                     mensagemPersonalizada & vbCrLf & vbCrLf & _
                     "Dados do Adiantamento:" & vbCrLf & _
                     "FILIAL: " & filial & vbCrLf & _
                     "SOLICITANTE: " & solicitante & vbCrLf & _
                     "CLIENTE: " & cliente & vbCrLf & _
                     "CÓDIGO: " & cod & vbCrLf & _
                     "VALOR: " & valor & vbCrLf & _
                     "DATA ADIANTAMENTO: " & dataAdiantamento & vbCrLf & _
                     "PREVISÃO NF: " & previsaoNF

        ' Configura e envia o e-mail
        With emailItem
            .To = listaEmails
            .CC = emailSecundario
            .Subject = "Cobrança de Adiantamento - " & cliente & " (" & cod & ")"
            .Body = corpoEmail
            .Send ' Pode usar .Display para testes
        End With

        ' Libera os objetos
        Set emailItem = Nothing
        Set objetoOutlook = Nothing

ProximaLinha:
        i = i + 1
    Loop

    MsgBox "Envio de e-mails finalizado com sucesso!", vbInformation

End Sub

'*************************************************************
' Função auxiliar para validar e-mails
'*************************************************************
Function IsValidEmail(email As String) As Boolean
    Dim re As Object
    Set re
